In [21]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re

from spacy.tokens import Token
from spacy.lang.en.stop_words import STOP_WORDS
from nltk.corpus import gutenberg, stopwords
from collections import Counter

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Bag of Words (BoW)
Count how many times each word appears in each sentence

In [2]:
# Utility function for standard text cleaning
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'. 
    text = re.sub(r'--',' ',text)
    # get rid of text between brackets
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [3]:
# Parse the cleaned novels. 
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [5]:
# Group into sentences
alice_sents = [[sent, 'Carroll'] for sent in alice_doc.sents]
persuasion_sents = [[sent, 'Austen'] for sent in persuasion_doc.sents]

# Combine the setnences from the two novels into one data frame
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [8]:
# Add function to make stop words case insensitive
stop_words_getter = lambda token: token.is_stop or token.lower_ in STOP_WORDS 
Token.set_extension('is_stop', getter=stop_words_getter)

In [13]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token._.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df


In [10]:
# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [14]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000


,year,correct,insolence,affliction,improve,busily,bow,oneself,action,relative,...,stage,benwick,occupy,bring,darkness,educate,ye,ancient,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


In [16]:
word_counts.to_csv('word_counts.csv')

# Random Forest

In [17]:
rfc = RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence', 'text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    Y, 
                                                    test_size=0.4, 
                                                    random_state=0)

train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Training set score: 0.9852664576802508

Test set score: 0.8839285714285714


# Logistic Regression

In [18]:
lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(3190, 3000) (3190,)
Training set score: 0.9504702194357367

Test set score: 0.9107142857142857


# Gradient Boosting

In [22]:
clf = GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.8855799373040752

Test set score: 0.8731203007518797


# Challenge 0:
Try to improve model performance. Other modeling techniques, more features, sentence-level features, contextual information, etc.

In [24]:
word_counts.head()

,year,correct,insolence,affliction,improve,busily,bow,oneself,action,relative,...,stage,benwick,occupy,bring,darkness,educate,ye,ancient,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


In [43]:
word_counts = pd.read_csv('word_counts.csv')

In [46]:
word_counts.head()

,Unnamed: 0,year,correct,insolence,affliction,improve,busily,bow,oneself,action,...,stage,benwick,occupy,bring,darkness,educate,ye,ancient,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Alice was beginning to get very tired of sitti...,Carroll
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,So she was considering in her own mind (as wel...,Carroll
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,There was nothing so VERY remarkable in that; ...,Carroll
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Oh dear!,Carroll
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,I shall be late!',Carroll


In [35]:
example_sentence = word_counts['text_sentence'][0]

# Look at some metrics around this sentence.
example_words = [token for token in example_sentence if not token.is_punct]
unique_words = set([token.text for token in example_words])

print(("There are {} words in this sentence, and {} of them are"
       " unique.").format(len(example_words), len(unique_words)))


There are 57 words in this sentence, and 41 of them are unique.


In [44]:
def sentence_words(example_sentence):
    return [token for token in example_sentence if not token.is_punct]

def sentence_unique(example_words):
    return set([token.text for token in example_words])

def sentence_length(example_words):
    return len(example_words)
    

In [45]:
word_counts['words'] = word_counts['text_sentence'].apply(sentence_words)

AttributeError: 'str' object has no attribute 'is_punct'

In [40]:
word_counts['len_words'] = word_counts['words'].apply(sentence_length)

In [42]:
word_counts['unique_words'] = word_counts['words'].apply(sentence_words)
word_counts['len_unique'] = words_counts['unique_words'].apply(sentence_length)

NameError: name 'words_counts' is not defined

In [41]:
word_counts.head()

,year,correct,insolence,affliction,improve,busily,bow,oneself,action,relative,...,occupy,bring,darkness,educate,ye,ancient,text_sentence,text_source,words,len_words
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll,"[Alice, was, beginning, to, get, very, tired, ...",57
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll,"[So, she, was, considering, in, her, own, mind...",56
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll,"[There, was, nothing, so, VERY, remarkable, in...",29
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"(Oh, dear, !)",Carroll,"[Oh, dear]",2
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll,"[I, shall, be, late]",4


# Challenge 1:
See if new model is as good at identifying Alice in Wonderland vs. any other work, or Persuasion vs. any other work, etc.